In [1]:
import numpy as np
import pickle
import pandas as pd
import CMR_IA as cmr
import sys
import scipy as sp

sys.path.append("../../Modeling/fitting")
from optimization_utils import make_boundary
from object_funcs import anal_perform_S2

## Load Stimuli and Semantic Matrix

In [2]:
# association recognition - cued recall
with open("simuS2_data/simuS2_design.pkl", "rb") as inp:
    df_study = pickle.load(inp)
    df_test = pickle.load(inp)
# df_study = df_study.loc[df_study.session < 300]  # for testing
# df_test = df_test.loc[df_test.session < 300]

In [3]:
df_study

,study_itemno1,study_itemno2,study_item1,study_item2,pair_idx,list,session,subject
0,809,1001,LEG,PAGE,16,0,0,0
1,489,1310,DOUGH,SIREN,2,0,0,0
2,1232,731,SANDWICH,ICEBERG,12,0,0,0
3,96,259,BASKETBALL,CATERPILLAR,9,0,0,0
4,1232,731,SANDWICH,ICEBERG,12,0,0,0
...,...,...,...,...,...,...,...,...
134995,1480,1422,TOOL,SWITCH,62,2,299,299
134996,1166,613,RANCH,FUNERAL,70,2,299,299
134997,1480,1422,TOOL,SWITCH,62,2,299,299
134998,1166,613,RANCH,FUNERAL,70,2,299,299


In [4]:
df_test

,test_itemno1,test_itemno2,test_item1,test_item2,correct_ans,pair_idx,type,test,list,session,subject
0,454,943,DETECTIVE,NEPTUNE,0,-1,extra,1,0,0,0
1,329,-1,CLOVE,None,1,10,Item_Pair,1,0,0,0
2,1084,-1,PLAQUE,None,1,20,Different_Item,1,0,0,0
3,484,671,DONKEY,GULL,0,-1,extra,1,0,0,0
4,1452,-1,TERRITORY,None,0,-1,extra,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
107995,957,818,NOSE,LIEUTENANT,0,-1,extra,2,2,299,299
107996,954,347,NITROGEN,COLOGNE,1,75,Item_Pair,2,2,299,299
107997,427,-1,CURTAIN,None,1,78,Pair_Item,2,2,299,299
107998,820,1567,LIGHTNING,VILLAIN,0,-1,extra,2,2,299,299


In [5]:
s_mat = np.load("../wordpools/ltp_FR_similarity_matrix.npy")

## Run CMR-IA

In [6]:
# define parameters
params = cmr.make_default_params()
params.update(learn_while_retrieving=True)

In [7]:
# pso results
_, _, what_to_fit = make_boundary(sim_name="S2")
optim_params = np.loadtxt("simuS2_data/S2_240722_200-200.txt")
for pname, pvalue in zip(what_to_fit, optim_params):
    params[pname] = pvalue
params.update(use_new_context=True)
params

{'beta_enc': 0.499972,
 'beta_rec': 0.5,
 'beta_cue': 0.381021,
 'beta_rec_post': 0.607876,
 'beta_distract': 0.112444,
 'phi_s': 2,
 'phi_d': 0.5,
 's_cf': 0,
 's_fc': 0.423607,
 'kappa': 0.5,
 'eta': 0.5,
 'omega': 5,
 'alpha': 1,
 'lamb': 0.5,
 'c_thresh': 0.5,
 'c_thresh_itm': 0.698477,
 'c_thresh_assoc': 0.740558,
 'd_assoc': 1,
 'rec_time_limit': 60000.0,
 'dt': 10,
 'nitems_in_accumulator': 50,
 'max_recalls': 50,
 'learn_while_retrieving': True,
 'use_new_context': True,
 'psi_s': 0,
 'psi_c': 1,
 'c_s': 0,
 'thresh_sigma': 0.00466,
 'ban_recall': None,
 'var_enc': 1,
 'bad_enc_ratio': 1,
 'a': 2800,
 'b': 20,
 'gamma_fc': 0.142263,
 'gamma_cf': 0.5}

In [8]:
# run the model
df_simu, f_in_acc, f_in_dif = cmr.run_success_multi_sess(params, df_study, df_test, s_mat, mode="Recog-Recog")

# merge to get more info
df_simu["test"] = df_test["test"]
df_simu = df_simu.merge(df_test, on=["session", "list", "test", "test_itemno1", "test_itemno2"])
df_simu

100%|██████████| 300/300 [02:45<00:00,  1.82it/s]

CMR Time: 165.14334893226624


,session,list,test_itemno1,test_itemno2,s_resp,s_rt,csim,test,test_item1,test_item2,correct_ans,pair_idx,type,subject
0,0,0,454,943,0.0,3.318627,0.361586,1,DETECTIVE,NEPTUNE,0,-1,extra,0
1,0,0,329,-1,1.0,718.333862,0.766499,1,CLOVE,None,1,10,Item_Pair,0
2,0,0,1084,-1,1.0,626.057373,0.773374,1,PLAQUE,None,1,20,Different_Item,0
3,0,0,484,671,0.0,1974.501465,0.715942,1,DONKEY,GULL,0,-1,extra,0
4,0,0,1452,-1,0.0,2.058532,0.337708,1,TERRITORY,None,0,-1,extra,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107995,299,2,957,818,0.0,125.932297,0.543396,2,NOSE,LIEUTENANT,0,-1,extra,299
107996,299,2,954,347,1.0,376.994415,0.798734,2,NITROGEN,COLOGNE,1,75,Item_Pair,299
107997,299,2,427,-1,1.0,1661.407471,0.724575,2,CURTAIN,None,1,78,Pair_Item,299
107998,299,2,820,1567,0.0,254.094147,0.578494,2,LIGHTNING,VILLAIN,0,-1,extra,299


In [9]:
# get correctness and condition
df_simu["correct"] = df_simu.s_resp == df_simu.correct_ans


def get_cond(x):
    this_type = x["type"]
    target = x["correct_ans"]
    if target == 1:
        if this_type == "Different_Item":
            return "Different_Item"
        elif this_type == "Item_Pair":
            return "Item_Pair"
        elif this_type == "Pair_Item":
            return "Pair_Item"
        elif this_type == "Same_Item":
            return "Same_Item"
        elif this_type == "Intact_Pair":
            return "Intact_Pair"
    elif target == 0:
        if this_type == "extra":
            return "NR_Lure"
        elif this_type == "Same_Item" or this_type == "Intact_Pair":
            return "Repeated_Lure"
        else:
            return "Discard"


df_simu["condition"] = df_simu.apply(get_cond, axis=1)
df_simu

,session,list,test_itemno1,test_itemno2,s_resp,s_rt,csim,test,test_item1,test_item2,correct_ans,pair_idx,type,subject,correct,condition
0,0,0,454,943,0.0,3.318627,0.361586,1,DETECTIVE,NEPTUNE,0,-1,extra,0,True,NR_Lure
1,0,0,329,-1,1.0,718.333862,0.766499,1,CLOVE,None,1,10,Item_Pair,0,True,Item_Pair
2,0,0,1084,-1,1.0,626.057373,0.773374,1,PLAQUE,None,1,20,Different_Item,0,True,Different_Item
3,0,0,484,671,0.0,1974.501465,0.715942,1,DONKEY,GULL,0,-1,extra,0,True,NR_Lure
4,0,0,1452,-1,0.0,2.058532,0.337708,1,TERRITORY,None,0,-1,extra,0,True,NR_Lure
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107995,299,2,957,818,0.0,125.932297,0.543396,2,NOSE,LIEUTENANT,0,-1,extra,299,True,NR_Lure
107996,299,2,954,347,1.0,376.994415,0.798734,2,NITROGEN,COLOGNE,1,75,Item_Pair,299,True,Item_Pair
107997,299,2,427,-1,1.0,1661.407471,0.724575,2,CURTAIN,None,1,78,Pair_Item,299,True,Pair_Item
107998,299,2,820,1567,0.0,254.094147,0.578494,2,LIGHTNING,VILLAIN,0,-1,extra,299,True,NR_Lure


In [10]:
df_simu.to_pickle("simuS2_data/simuS2_result.pkl")

## Analysis

In [11]:
with open("simuS2_data/simuS2_result.pkl", "rb") as inp:
    df_simu = pickle.load(inp)

In [12]:
subjects = np.unique(df_simu.subject)
stats = []
for subj in subjects:
    df_subj = df_simu.query(f"subject=={subj} and list % 3 != 0")  # discard first list
    stats_subj = anal_perform_S2(df_subj)
    stats.append(stats_subj)
stats_mean = np.nanmean(stats, axis=0)
stats_mean.round(2)

array([[0.78, 0.77, 0.27],
       [0.78, 0.88, 0.61],
       [0.88, 0.83, 0.64],
       [0.79, 0.82, 0.81],
       [0.88, 0.91, 0.81],
       [0.1 , 0.13, 0.61],
       [0.09, 0.09, 0.  ]])

In [13]:
stats_mean

array([[0.77592172, 0.77486845, 0.26940755],
       [0.78453437, 0.88043817, 0.60695663],
       [0.87534055, 0.83378535, 0.63852395],
       [0.78517665, 0.82253187, 0.81015222],
       [0.87983333, 0.90566667, 0.81183312],
       [0.09899792, 0.13064775, 0.61037954],
       [0.09144444, 0.08816667, 0.        ]])

In [14]:
ground_truth = np.array(
    [
        [0.82, 0.68, 0.26],  # diff item
        [0.82, 0.85, 0.64],  # item/pair
        [0.91, 0.85, 0.59],  # pair/item
        [0.81, 0.82, 0.86],  # same item
        [0.90, 0.92, 0.94],  # intact pair
        [0.07, 0.15, 0.54],  # repeated lure
        [0.07, 0.06, 0],  # non-repeated lure
    ]
)
err = np.mean(np.power(stats_mean - ground_truth, 2))
err

0.0021759302375999783

In [15]:
print(sp.stats.sem(np.array(stats), axis=0, nan_policy="omit").round(3))

[[0.008 0.007 0.028]
 [0.006 0.005 0.02 ]
 [0.006 0.007 0.018]
 [0.008 0.007 0.012]
 [0.009 0.007 0.008]
 [0.011 0.012 0.018]
 [0.002 0.002 0.   ]]
